<a href="https://colab.research.google.com/github/minanalysis/review-liar_nlp_pjt/blob/main/%EC%83%98%ED%94%8C%EB%A7%81_%EA%B5%B0%EC%A7%91%EC%84%A0%EC%A0%95_%EC%9C%84%ED%95%9C_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%B6%94%EC%B6%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

예시 : 샘플링 군집 추출을 위한 요인 2개 (장르,기간) 중 장르의 공포/스릴러

## 데이터 프레임 자체 기본 전처리 

In [ ]:
import pandas as pd
import numpy as np 
import io


In [ ]:
from google.colab import files
myfile = files.upload()

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
df_horror= pd.read_csv(io.BytesIO(myfile['df_horror.csv']))
df_horror

In [ ]:
df_horror['댓글']=df_horror['댓글'].str.replace('[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]','') #한글이 아닌 문자들은 모두 제거 

In [ ]:
df_horror['댓글']=df_horror['댓글'].replace('', np.nan)

In [ ]:
df_horror=df_horror.dropna() # 결측행 제거 

In [ ]:
df_horror=df_horror.sample(50000)
df_horror# 랜덤으로 샘플 5만개 추출 

In [ ]:
df_horror=df_horror.reset_index() # 인덱스 번호 재부여 

## 텐서플로우 및 필요한 라이브러리, 형태소 분석기 패키지들 불러오기 

In [ ]:
! pip install konlpy 

In [ ]:
from konlpy.tag import * 

okt = Okt()


In [ ]:
!pip install tensorflow 

In [ ]:
!pip install keras

In [ ]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

## 댓글 전처리 

### 띄어쓰기 교정 

In [ ]:
from pykospacing import Spacing

In [ ]:
spacing=Spacing()

In [ ]:
from tqdm import trange

In [ ]:
horror_review_list = [] # 5만개의 리뷰( 리뷰 당 한 문서로 간주 ) 를 띄어쓰기 교정해서 리스트에 담기 
                       # 추후에, 이 리스트 값으로 댓글 컬럼을 바꿔줄거임. 후, 토큰화 하여 새컬럼에 담아줄 것 
for i in trange(len(df_horror['댓글'])):
  horror_review_list.append(spacing(df_horror['댓글'][i]))

In [ ]:
df_horror['댓글']=horror_review_list # 데이터프레임의 댓글컬럼을 띄어쓰기 교정한 문장으로 변경시켜줌 
df_horror.to_csv('df_horror_save1.csv',index=False) #혹시몰라 중간저장

### 단어 토큰화

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
word_tokens=[] 
for sentence in df_horror['댓글']:
    word_tokens.append(okt.morphs(sentence)) # 형태소 분석기 이용 토큰화

# word_tokens 라는 리스트의 형식 자체가, 한 문장 당 토큰화된 단어들이 리스트로 들어가고 , 
#그 '리스트'가 word_tokens라는 리스트의 값이 되는 것. 즉, 리스트를 값으로 가지는 리스트! 

In [ ]:
len(word_tokens)

In [ ]:
tokenizer=Tokenizer() # 텐서플로우의 케라스 패키지의 토크나이저 불러온 것

In [ ]:
tokenizer.fit_on_texts(word_tokens) # 토크나이저가 알아서 정수 인코딩을 해줌 ( 빈도수 높은게 낮은 인덱싱 ) 
# 지금 우리 단계에서는 정수 인코딩은 필요 없지만, 빈도수를 내주기 위해서 실행해줘야 함 

In [ ]:
horror_freq=sorted(tokenizer.word_counts.items(), key= lambda x:x[1], reverse=True) 
# tokenizer.word_counts 는 각 단어당 빈도수를 보여줌. 딕셔너리 형식이라 빈도수가 높은대로 정렬하기 위해 람다 사용 
# 우리는 워드클라우드 사용한다면, 사실상 필요없는 단계지만 육안으로 확인해봄 

## 워드클라우드 위한 작업 

### 토큰화 단어들 합치기 

In [ ]:
sum_words= sum(word_tokens,[])
# 문장 당 토큰화된 단어들이 각각의 리스트로 들어간 것을 한 리스트로 모두 합쳐버리기 
# 워드클라우드가 넣기 위해 이건 중복단어 같은 게 전혀 처리 안된, 단순히 한 리뷰 당 토큰화된 단어들의 집합임! 단어들이 빈도가 여러개인게 당연! 

In [ ]:
sum_words

In [ ]:
total_words = [] # 최종 단어집합 

for i in sum_words:
  if len(i) > 1:
    total_words.append(i)
  else:
    continue

# 단어 길이가 1 이하인 것은 정제
# 길이와 불용어에 대한건 내일 얘기하면서 정의 해보기로 했음

In [ ]:
total_words_df=pd.DataFrame()

In [ ]:
total_words_df['total_words']=total_words

In [ ]:
total_words_df.to_csv('total_words_df2.csv',index=False)

### 불용어 제거 

In [ ]:
stop_word = ['지', '임', '게', '하', '고', '을', '인', '듯', '의', '가', '이', '영화','은', '들', '는', '좀', '잘', '걍', '과', '와', '도', '를', '으로', '나','자', '에', '와', '네', '한', '하다', '았', '앗', '엇', '주', '되', '다', '따', '영','이', '걸', '만','즐','엔', '임', '후','왠','만','할','로','펌','무','적','원','치', '류','옆','타','함','구','호','수','못','랑','요','씨','보고','대','됨','번','테','력','박','습','라','데','옴','오','기','줌','않','함','그','거']

In [ ]:
total_words2 = [] # 최종 단어집합 

for i in sum_words:
    if i not in stop_word:
        total_words2.append(i)
    else:
        continue

In [ ]:
final_words_df2=pd.DataFrame()

In [ ]:
final_words_df2['토큰단어']=total_words2

In [ ]:
final_words_df2.to_csv('final_words_df2.csv',index=False)

### 토큰화된 단어들 컬럼으로 넣기 

In [ ]:
#불용어 처리 안한 단어들 , 우리가 사용할 건 위의 불용어 제거 csv

df_horror['토큰단어']=word_tokens

# 이건 for 문 돌려서 컬럼에 들어가는 값들을 리스트(['단어1','단어2','단어3'])가 아니라,
# 그냥 ('단어1','단어2','단어3') 넣으려고 했는데 
# 나중에 활용을 생각했을 때도 리스트로 넣어두는 것이 낫지 않을까 싶어서 그냥 넣었습니다!
# 따라서, 컬럼이 기본 크롤링 함수로 인한 컬럼 + 영화제목 + 토큰단어 인 데이터프레임이나 위의 
# total_words 리스트 중 하나를 내일 들고오심 될 것 같습니다 

In [ ]:
df_horror.to_csv('df_horror_save2.csv',index=False)